# Getting started with spike-field coherence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: a randomly spiking neuron

To start, let's create a fake spike train for a randomly spiking neuron, and compute the autocovariance and spectrum.

In [ ]:
N  = 1000;                          # Number of bins.                   
dt = 0.001;                         # Duration of each bin.
T  = N*dt;                
tm = np.arange(0,N)*dt;

lambda0 = 0.01                      # Average firing rate.
dn = np.random.poisson(lambda0,N)   # Create the spike train as "coin flips"

plt.plot(tm, dn)                    # Plot it.
plt.xlabel('Time [s]');

In [ ]:
# Compute the autocovariance.

ac_xx = 1 / N * np.correlate(dn-dn.mean(),dn-dn.mean(), 'full')
lags = np.arange(-N + 1, N)                  # Create a lag axis,
plt.plot(lags * dt, ac_xx)                   # ... and plot the result.
plt.xlabel('Lag [s]')
plt.ylabel('Autocovariance');

In [ ]:
# Compute the spectrum.

Ia = np.fft.fft(dn - dn.mean())            # Compute the FT,
Ja = 2 * dt**2 / T * (Ia * Ia.conj())      # ... and the spectrum.

f = fftfreq(N, dt)                         # Create frequency axis.

plt.plot(f, np.real(Ja))                   # Plot the spectrum.
plt.plot(f,2*dt*lambda0*np.ones(N), 'b')   # And our guess from in-class analysis.
plt.xlabel('Frequency [Hz]');

# Analyze an example data set

In [ ]:
# Load modules we'll need.
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Load the data and look at it.

**Q.** Do you observe evidence of cross-frequency coupling?

### Conclusions

* 
* 

In [ ]:
# Load the data.
data = sio.loadmat('spikes-LFP-1.mat')       # Load the multiscale data,
y = data['y']                                # ... get the LFP data,
n = data['n']                                # ... get the spike data,
t = data['t'].reshape(-1)                    # ... get the time axis,
K = np.shape(n)[0]                           # Get the number of trials,
N = np.shape(n)[1]                           # ... and the number of data points in each trial,
dt = t[1]-t[0]                               # Get the sampling interval.

In [ ]:
# And plot an example.
plt.plot(t, y[0,:])
plt.plot(t, n[0,:]);

## Step 2: Compute the trial-averaged (auto-)spectrum for each data type.

**Q.** What rhythms are present in the data?

### Conclusions

* 
* 

In [ ]:
SYY = np.zeros(int(N/2+1))                                       # Variable to store field spectrum.
SNN = np.zeros(int(N/2+1))                                       # Variable to store spike spectrum.
SYN = np.zeros(int(N/2+1), dtype=complex)                        # Variable to store cross spectrum.

for k in np.arange(K):                                           # For each trial,
    yf = np.fft.rfft((y[k,:]-np.mean(y[k,:])) *np.hanning(N))    # Hanning taper the field,
    nf = np.fft.rfft((n[k,:]-np.mean(n[k,:])))                   # ... but do not taper the spikes.
    SYY = SYY + ( np.real( yf*np.conj(yf) ) )/K                  # Field spectrum
    SNN = SNN + ( np.real( nf*np.conj(nf) ) )/K                  # Spike spectrum
    SYN = SYN + (          yf*np.conj(nf)   )/K                  # Cross spectrum

f = np.fft.rfftfreq(N, dt)                                       # Frequency axis for plotting

plt.plot(f,10*np.log10(SYY))                             # Plot the result.
plt.plot(f,10*np.log10(SNN))
plt.xlim([0, 100])
plt.ylim([0, 50])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power');

## Step 3: Compute the coherence between the two signals

**Q.** What do you find? Is there evidence of coherence between the two signals?

**Q.** How do the coherence results compare to the spectral results?

### Conclusions

* 
* 

In [ ]:
cohr = np.real(SYN*np.conj(SYN)) / SYY / SNN # Spike-field coherence (computed all parts above)

plt.plot(f,cohr)                             # Plot the result.
plt.xlim([0, 100])
plt.ylim([0, 1])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Coherence');